# Loop 4 Analysis: Understanding the Gap and Next Steps

## Key Observations:
1. All 4 experiments stuck at 70.676102
2. Random initialization without optimization produces much worse results
3. Deletion cascade found 0 improvements
4. The baseline is at a very strong local optimum

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
import os

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def get_tree_polygon(x, y, deg):
    base_poly = Polygon(zip(TX, TY))
    rotated = affinity.rotate(base_poly, deg, origin=(0, 0))
    translated = affinity.translate(rotated, x, y)
    return translated

def get_bounding_box_side(trees):
    if not trees:
        return float('inf')
    all_x = []
    all_y = []
    for x, y, deg in trees:
        poly = get_tree_polygon(x, y, deg)
        bounds = poly.bounds
        all_x.extend([bounds[0], bounds[2]])
        all_y.extend([bounds[1], bounds[3]])
    width = max(all_x) - min(all_x)
    height = max(all_y) - min(all_y)
    return max(width, height)

print('Functions defined')

Functions defined


In [2]:
# Load baseline and analyze per-N scores
baseline_path = '/home/code/experiments/001_baseline/santa-2025.csv'
df = pd.read_csv(baseline_path, dtype=str)

def load_all_configs(df):
    configs = {}
    for n in range(1, 201):
        prefix = f'{n:03d}_'
        rows = df[df['id'].str.startswith(prefix)]
        trees = []
        for _, row in rows.iterrows():
            x = float(str(row['x']).replace('s', ''))
            y = float(str(row['y']).replace('s', ''))
            deg = float(str(row['deg']).replace('s', ''))
            trees.append((x, y, deg))
        configs[n] = trees
    return configs

configs = load_all_configs(df)

# Calculate per-N scores
scores = {}
for n in range(1, 201):
    side = get_bounding_box_side(configs[n])
    scores[n] = side**2 / n

total = sum(scores.values())
print(f'Total score: {total:.6f}')
print(f'Target: 68.919154')
print(f'Gap: {total - 68.919154:.6f}')

Total score: 70.676102
Target: 68.919154
Gap: 1.756948


In [3]:
# Analyze which N values have the most room for improvement
# Compare to theoretical minimum (perfect packing efficiency)

print('\nPer-N score analysis (top 20 by contribution):')
print('=' * 60)

sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
for n, score in sorted_scores[:20]:
    side = get_bounding_box_side(configs[n])
    print(f'N={n:3d}: score={score:.6f}, side={side:.6f}')


Per-N score analysis (top 20 by contribution):
N=  1: score=0.661250, side=0.813173
N=  2: score=0.450779, side=0.949504
N=  3: score=0.434745, side=1.142031
N=  5: score=0.416850, side=1.443692
N=  4: score=0.416545, side=1.290806
N=  7: score=0.399897, side=1.673104
N=  6: score=0.399610, side=1.548438
N=  9: score=0.387415, side=1.867280
N=  8: score=0.385407, side=1.755921
N= 15: score=0.379203, side=2.384962
N= 10: score=0.376630, side=1.940696
N= 21: score=0.376451, side=2.811667
N= 20: score=0.376057, side=2.742469
N= 11: score=0.375736, side=2.033002
N= 22: score=0.375258, side=2.873270
N= 16: score=0.374128, side=2.446640
N= 26: score=0.373997, side=3.118320
N= 12: score=0.372724, side=2.114873
N= 13: score=0.372323, side=2.200046
N= 25: score=0.372144, side=3.050182


In [4]:
# Analyze score distribution by N ranges
ranges = [
    (1, 10, 'N=1-10'),
    (11, 50, 'N=11-50'),
    (51, 100, 'N=51-100'),
    (101, 150, 'N=101-150'),
    (151, 200, 'N=151-200')
]

print('\nScore distribution by N ranges:')
print('=' * 60)
for start, end, label in ranges:
    range_score = sum(scores[n] for n in range(start, end+1))
    pct = range_score / total * 100
    print(f'{label}: {range_score:.6f} ({pct:.1f}%)')


Score distribution by N ranges:
N=1-10: 4.329128 (6.1%)
N=11-50: 14.713045 (20.8%)
N=51-100: 17.641148 (25.0%)
N=101-150: 17.144118 (24.3%)
N=151-200: 16.848664 (23.8%)


In [5]:
# Check what's available in snapshots for better solutions
import glob

print('\nAvailable pre-optimized solutions in snapshots:')
print('=' * 60)

snapshot_csvs = glob.glob('/home/nonroot/snapshots/santa-2025/**/*.csv', recursive=True)
print(f'Total CSV files found: {len(snapshot_csvs)}')

# Check for any solutions with better scores
best_scores = {}
for csv_path in snapshot_csvs[:30]:  # Check first 30
    try:
        df_check = pd.read_csv(csv_path, dtype=str)
        if 'id' in df_check.columns and 'x' in df_check.columns:
            # Quick score check for N=1
            n1_rows = df_check[df_check['id'].str.startswith('001_')]
            if len(n1_rows) == 1:
                x = float(str(n1_rows.iloc[0]['x']).replace('s', ''))
                y = float(str(n1_rows.iloc[0]['y']).replace('s', ''))
                deg = float(str(n1_rows.iloc[0]['deg']).replace('s', ''))
                side = get_bounding_box_side([(x, y, deg)])
                score_n1 = side**2
                if score_n1 < 0.67:  # Good N=1 score
                    print(f'{os.path.basename(csv_path)}: N=1 score = {score_n1:.6f}')
    except:
        pass


Available pre-optimized solutions in snapshots:
Total CSV files found: 683
submission.csv: N=1 score = 0.661250
submission_best.csv: N=1 score = 0.661250
submission_v18.csv: N=1 score = 0.661250
submission.csv: N=1 score = 0.661250


submission_v21.csv: N=1 score = 0.661250


optimized.csv: N=1 score = 0.661250
submission.csv: N=1 score = 0.661250
candidate_000.csv: N=1 score = 0.661250
candidate_004.csv: N=1 score = 0.661250
candidate_002.csv: N=1 score = 0.661250
candidate_003.csv: N=1 score = 0.661250
candidate_001.csv: N=1 score = 0.661250


ensemble.csv: N=1 score = 0.661250
submission.csv: N=1 score = 0.661250
santa-2025.csv: N=1 score = 0.661250
best_ensemble.csv: N=1 score = 0.661250
72.49.csv: N=1 score = 0.661250
71.97.csv: N=1 score = 0.661250
72.49.csv: N=1 score = 0.661250


71.97.csv: N=1 score = 0.661250
submission_JKoT4.csv: N=1 score = 0.661250
New_Tree_144_196.csv: N=1 score = 0.661250
submission_JKoT3.csv: N=1 score = 0.661250
santa2025_ver2_v61.csv: N=1 score = 0.661250
submission_JKoT2.csv: N=1 score = 0.661250
santa2025_ver2_v67.csv: N=1 score = 0.661250
santa2025_ver2_v76.csv: N=1 score = 0.661250


submission_70_936673758122.csv: N=1 score = 0.661250
santa2025_ver2_v65.csv: N=1 score = 0.661250
submission_70_926149550346.csv: N=1 score = 0.661250


In [6]:
# Key insight: The gap of 1.76 points requires ~2.5% improvement
# This is significant and cannot be achieved by local optimization

print('\n=== KEY INSIGHTS ===')
print('=' * 60)
print(f'Current score: {total:.6f}')
print(f'Target score: 68.919154')
print(f'Gap: {total - 68.919154:.6f} ({(total - 68.919154) / 68.919154 * 100:.2f}%)')
print()
print('What has been tried and FAILED:')
print('  1. Baseline from pre-optimized solutions')
print('  2. Full ensemble from 30 CSVs')
print('  3. Deletion cascade (0 improvements)')
print('  4. Random initialization without optimization (much worse)')
print()
print('What MUST be tried next:')
print('  1. Lattice initialization + SA optimization (egortrushin kernel)')
print('  2. Use bbox3 C++ optimizer with fresh configurations')
print('  3. Parallel SA with fractional translation (seshurajup kernel)')
print('  4. Greedy backtracking (constructive approach)')
print()
print('CRITICAL: Random placement without optimization is useless.')
print('The baseline is the result of sophisticated optimization.')


=== KEY INSIGHTS ===
Current score: 70.676102
Target score: 68.919154
Gap: 1.756948 (2.55%)

What has been tried and FAILED:
  1. Baseline from pre-optimized solutions
  2. Full ensemble from 30 CSVs
  3. Deletion cascade (0 improvements)
  4. Random initialization without optimization (much worse)

What MUST be tried next:
  1. Lattice initialization + SA optimization (egortrushin kernel)
  2. Use bbox3 C++ optimizer with fresh configurations
  3. Parallel SA with fractional translation (seshurajup kernel)
  4. Greedy backtracking (constructive approach)

CRITICAL: Random placement without optimization is useless.
The baseline is the result of sophisticated optimization.
